In [1]:
import db_management
import db_manipulation as db
import extract_basic_institution_info
import find_events_page

In [ ]:
extract_basic_institution_info.extract_information()

In [ ]:
vzgojno_izobrazevalni_zavodi = db.get_all_vzgojno_izobrazevalni_zavodi()

for zavod in vzgojno_izobrazevalni_zavodi:
    events_page_url = find_events_page.find_events_page(zavod["website"])
    if not events_page_url: events_page_url = ""

    db.add_events_page_url(zavod["id"], events_page_url)